In [1]:
#####################################################
#
# APLICAR PCA A DATOS QUE NUNCA HA VISTO
#
#####################################################
# Deben cargarse los archivos
# - preprocessor_cat.joblib
# - pca_pipe_num.joblib
# - pca_metadata.json
# - T_train_final_objetivo.csv
# - csv de tus nuevos datos
# Devolverá T_new_final.csv: el csv de PCA aplicado a los nuevos datos

# === Cargar artefactos para inferencia ===
import joblib
import json
import pandas as pd
#####################################################
preprocessor_cat = joblib.load("../preprocessing/preprocessor_cat.joblib")
pca_pipe = joblib.load("../preprocessing/pca_pipe_num.joblib")
with open("../preprocessing/pca_metadata.json", "r") as f:
#####################################################
    meta = json.load(f)

cols_num = meta["cols_num"]
cols_cat = meta["cols_cat"]
pc_cols  = meta["pc_cols"]
cat_out_cols = meta["cat_out_cols"]

# PCA DE ENTRENAMIENTO CON OBJETIVO
#####################################################
entrenamiento = pd.read_csv("../preprocessing/T_train_final_objetivo.csv")   ### todas las pca, categoricas y objetivo
print(entrenamiento)
#####################################################
entrenamiento_pca_objetivo = entrenamiento[pc_cols]


          PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0   -1.477137  1.691553  0.957352  0.513384 -1.126519  0.212635 -0.455387   
1   -2.171149 -1.557130 -0.087657  1.356527  0.310034 -0.319167 -0.227629   
2    3.105425  1.459959 -0.324170 -1.422175  0.602097  0.053437 -0.302450   
3    0.731003  0.777701  1.306709  0.492109  0.769146  1.659223 -0.659452   
4   -2.454057 -1.050413 -0.752110 -0.758958  0.663332  1.089412 -0.062015   
..        ...       ...       ...       ...       ...       ...       ...   
322  3.646022 -0.512246  0.231740  1.304080  0.954428  1.398180 -0.503080   
323 -0.769214  1.354167  0.803113  0.873207 -1.117007  0.165526  0.121606   
324  3.659812 -0.415579  0.993515  0.754257  1.868428  0.400782  1.108959   
325 -3.297338 -1.012676 -1.675564  0.312927  0.060905  1.200612 -0.165040   
326  2.675409 -0.470792  0.614622 -0.308239  0.765606 -0.671973 -0.964462   

     sex___1  sex___2  objetivo  
0        1.0      0.0       118  
1      

In [2]:
# NUEVOS DATOS
#####################################################
new_df = pd.read_csv("../../data/diabetes_desconocido.csv")

#####################################################
X_new_num = new_df[cols_num]
X_new_cat = new_df[cols_cat]

# Categóricas (mismo encoder, sin re-ajustar)
X_new_cat_proc = preprocessor_cat.transform(X_new_cat)
df_new_cat_encode = pd.DataFrame(X_new_cat_proc, columns=cat_out_cols, index=new_df.index)

# Numéricas → (preproc_num + StdScaler + PCA) con el pipeline guardado
T_new = pca_pipe.transform(X_new_num)
T_new_df = pd.DataFrame(T_new, columns=pc_cols, index=new_df.index)

# Final
T_new_final = pd.concat([T_new_df, df_new_cat_encode], axis=1)
T_new_final.to_csv("T_new_final.csv",index=False)

In [3]:
import plotly.express as px
import plotly.graph_objects as go

# --- 1) figura base: train coloreado por objetivo (igual que antes) ---
y_train = entrenamiento["objetivo"]
df_plot2 = entrenamiento_pca_objetivo.iloc[:, :2].copy()
df_plot2["objetivo"] = y_train.loc[entrenamiento_pca_objetivo.index].astype(str)

fig = px.scatter(
    df_plot2, x="PC1", y="PC2", color="objetivo",
    opacity=0.85, title="PC1 vs PC2 — Train (objetivo) + Nuevos",
    height=550
)
fig.update_traces(marker=dict(size=6))

# --- 2) overlay: nuevos (un solo trazo, símbolo distinto) ---
new2 = T_new_df.loc[:, ["PC1", "PC2"]].copy()
fig.add_trace(
    go.Scatter(
        x=new2["PC1"], y=new2["PC2"],
        mode="markers",
        name="nuevos",
        marker=dict(size=9, line=dict(width=1)),
        opacity=0.95,
        showlegend=True
    )
)

fig.update_layout(xaxis_title="PC1", yaxis_title="PC2")
fig.show()

In [4]:

T_new_final


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,sex___1,sex___2
0,1.035769,-0.767906,0.874265,0.294765,-0.463500,0.425297,0.222057,0.0,1.0
1,1.058362,1.369898,-1.560936,0.350316,0.794935,-1.275209,0.169949,0.0,1.0
2,-0.423913,-0.672876,0.489503,0.912180,-0.123744,-0.630890,0.874674,0.0,1.0
3,0.850003,-0.264890,-1.109337,-0.885740,-0.515039,0.595948,-0.535683,1.0,0.0
4,-1.662149,3.878706,0.287365,-1.073054,1.591310,0.144845,-0.928915,1.0,0.0
